In [1]:
import os
import sys
import time
import math
import tqdm
import imageio
import pathlib
import dataclasses
import collections
import numpy as np
import torch
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from datasets import load_dataset
from openpi.training import config as _config
from openpi.policies import policy_config
from openpi_client import image_tools

sys.path.append("/home/wyz/openpi/third_party/libero")
from libero.libero import benchmark, get_libero_path
from libero.libero.envs import OffScreenRenderEnv

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["JAX_PLATFORMS"] = "cpu"
os.environ["TORCH_DISABLE_DYNAMO"] = "1"
os.environ["PYTORCH_JIT"] = "0"

LIBERO_DUMMY_ACTION = [0.0] * 6 + [-1.0]
LIBERO_ENV_RESOLUTION = 256


# ------------------------------------------------------------
# ✅ 加载策略
# ------------------------------------------------------------
def load_policy():
    print("Loading policy checkpoint ...")
    config = _config.get_config("pi05_libero")
     # 检查 ToMe 配置
    print(f"\n[ToMe Config Check]")
    print(f"  Config type: {type(config.model)}")
    print(f"  tome_enabled: {getattr(config.model, 'tome_enabled', 'NOT FOUND')}")
    print(f"  tome_ratio: {getattr(config.model, 'tome_ratio', 'NOT FOUND')}")
    print(f"  tome_metric: {getattr(config.model, 'tome_metric', 'NOT FOUND')}")
    
    ckpt = "/home/wyz/openpi/checkpoints/pi05_libero_pytorch.pt"

    def _no_compile(x, *args, **kwargs):
        return x
    torch.compile = _no_compile

    try:
        torch.jit._state.disable()
    except Exception:
        pass

    policy = policy_config.create_trained_policy(config, ckpt, pytorch_device="cuda:0")
    print("✅ Policy loaded:", type(policy))
    if hasattr(policy, "model"):
        print(" → Using model:", policy.model.__class__.__module__)
    return policy


# ------------------------------------------------------------
# ✅ 工具函数
# ------------------------------------------------------------
def _quat2axisangle(quat):
    quat = np.array(quat)
    quat = np.clip(quat, -1.0, 1.0)
    den = np.sqrt(max(1e-6, 1.0 - quat[3] ** 2))
    if math.isclose(den, 0.0):
        return np.zeros(3)
    return (quat[:3] * 2.0 * math.acos(quat[3])) / den


def _get_libero_env(task, resolution, seed):
    task_desc = task.language
    bddl_path = pathlib.Path(get_libero_path("bddl_files")) / task.problem_folder / task.bddl_file
    env_args = {"bddl_file_name": bddl_path, "camera_heights": resolution, "camera_widths": resolution}
    env = OffScreenRenderEnv(**env_args)
    env.seed(seed)
    return env, task_desc


# ------------------------------------------------------------
# ✅ 参数类
# ------------------------------------------------------------
@dataclasses.dataclass
class Args:
    task_suite_name: str = "libero_spatial"
    num_trials_per_task: int = 1       # ✅ 每个任务只跑一次
    num_steps_wait: int = 10
    replan_steps: int = 5
    resize_size: int = 224
    video_out_path: str = "./libero_videos_prune"
    seed: int = 7


# ------------------------------------------------------------
# ✅ 主评估函数
# ------------------------------------------------------------
def eval_libero_prune(args: Args):
    policy = load_policy()
    np.random.seed(args.seed)

    # ----------------------
    print("🔥 Warming up policy (compile + autotune)...")
    dummy_element = {
        "observation/image": np.zeros((args.resize_size, args.resize_size, 3), dtype=np.uint8),
        "observation/wrist_image": np.zeros((args.resize_size, args.resize_size, 3), dtype=np.uint8),
        "observation/state": np.zeros(8, dtype=np.float32),
        "prompt": "dummy task",
    }
    with torch.inference_mode():
        _ = policy.infer(dummy_element)
    torch.cuda.synchronize()
    print("✅ Warmup done, start evaluation...\n")

    # ----------------------
    benchmark_dict = benchmark.get_benchmark_dict()
    task_suite = benchmark_dict[args.task_suite_name]()
    num_tasks = task_suite.n_tasks

    pathlib.Path(args.video_out_path).mkdir(parents=True, exist_ok=True)

    # 最大步数设定
    if args.task_suite_name == "libero_spatial":
        max_steps = 220
    elif args.task_suite_name == "libero_object":
        max_steps = 280
    elif args.task_suite_name == "libero_goal":
        max_steps = 300
    else:
        max_steps = 400

    total_episodes, total_successes = 0, 0
    total_infer_times = []

    print(f"\n🔹 Running evaluation on suite: {args.task_suite_name} ({num_tasks} tasks)")

    for task_id in tqdm.tqdm(range(num_tasks), desc="Tasks"):
        task = task_suite.get_task(task_id)
        init_states = task_suite.get_task_init_states(task_id)
        init_states = init_states[:args.num_trials_per_task]

        env, desc = _get_libero_env(task, LIBERO_ENV_RESOLUTION, args.seed)
        task_episodes, task_successes = 0, 0
        task_infer_times = []

        for ep in range(args.num_trials_per_task):
            obs = env.set_init_state(init_states[ep])
            env.reset()
            action_plan = collections.deque()
            replay_imgs = []

            # 环境稳定等待
            for _ in range(args.num_steps_wait):
                obs, _, _, _ = env.step(LIBERO_DUMMY_ACTION)

            done = False
            t = 0
            while t < max_steps + args.num_steps_wait:
                try:
                    img = np.ascontiguousarray(obs["agentview_image"][::-1, ::-1])
                    wrist = np.ascontiguousarray(obs["robot0_eye_in_hand_image"][::-1, ::-1])

                    img = image_tools.convert_to_uint8(image_tools.resize_with_pad(img, args.resize_size, args.resize_size))
                    wrist = image_tools.convert_to_uint8(image_tools.resize_with_pad(wrist, args.resize_size, args.resize_size))
                    replay_imgs.append(img)

                    if not action_plan:
                        element = {
                            "observation/image": img,
                            "observation/wrist_image": wrist,
                            "observation/state": np.concatenate(
                                (obs["robot0_eef_pos"], _quat2axisangle(obs["robot0_eef_quat"]), obs["robot0_gripper_qpos"])
                            ),
                            "prompt": str(desc),
                        }

                        # ✅ 纯推理时间测量
                        with torch.inference_mode():
                            torch.cuda.synchronize()
                            t0 = time.perf_counter()
                            actions = policy.infer(element)["actions"]
                            torch.cuda.synchronize()
                            infer_time = (time.perf_counter() - t0) * 1000
                            task_infer_times.append(infer_time)

                        action_plan.extend(actions[: args.replan_steps])

                    action = action_plan.popleft()
                    obs, reward, done, info = env.step(action.tolist())

                    if done:
                        task_successes += 1
                        total_successes += 1
                        break

                    t += 1

                except Exception as e:
                    print(f"[⚠️] Exception in task {task_id}, ep {ep}: {e}")
                    break

            task_episodes += 1
            total_episodes += 1

            tag = "success" if done else "failure"
            seg = desc.replace(" ", "_")
            imageio.mimwrite(
                pathlib.Path(args.video_out_path) / f"rollout_{seg}_{tag}_ep{ep}.mp4",
                [np.asarray(x) for x in replay_imgs],
                fps=10,
            )

        # ===== 每个任务结果 =====
        sr = 100 * task_successes / max(task_episodes, 1)
        total_sr = 100 * total_successes / max(total_episodes, 1)
        avg_infer = np.mean(task_infer_times)
        total_infer_times.extend(task_infer_times)

        print(f"✅ Task {task_id}: {desc}")
        print(f" Success rate: {task_successes}/{task_episodes} = {sr:.1f}%")
        print(f" Avg inference time: {avg_infer:.2f} ms")
        print(f" Overall success: {total_successes}/{total_episodes} = {total_sr:.1f}%\n")

    # ===== 汇总结果 =====
    print("=" * 60)
    print(f"🎯 Final success rate on {args.task_suite_name}: {total_successes}/{total_episodes} = {(100 * total_successes / total_episodes):.2f}%")
    print(f"⚙️ Avg inference time over all tasks: {np.mean(total_infer_times):.2f} ± {np.std(total_infer_times):.2f} ms")
    print("=" * 60)


# ------------------------------------------------------------
# ✅ 运行入口
# ------------------------------------------------------------
if __name__ == "__main__":
    args = Args(task_suite_name="libero_spatial", num_trials_per_task=1)  # ✅ 每个任务只跑一次
    with torch.autocast(device_type='cuda', dtype=torch.bfloat16):
        eval_libero_prune(args)


/home/wyz/miniconda3/envs/pi0/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[robosuite WARNING] No private macro file found! (__init__.py:7)
[robosuite WARNING] It is recommended to use a private macro file (__init__.py:8)
[robosuite WARNING] To setup, run: python /home/wyz/miniconda3/envs/pi0/lib/python3.11/site-packages/robosuite/scripts/setup_macros.py (__init__.py:9)
Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.

Loading policy checkpoint ...

[ToMe Config Check]
  Config type: <class 'openpi.models.pi0_config.Pi0Config'>
  tome_enabled: True
  tome_ratio: 0.1
  tome_metric: cosine
[ToMe Debug] SiglipVisionTransformer.__init__ called
[ToMe Debug] config type: <class 'transformers.models.siglip.configuration_siglip.SiglipVisionConfig'>
[ToMe Debug] hasattr(config, 'tome_enabled'): True
[ToMe Debug] config.tome_enabled = True
[ToMe Debug] config.tome_ratio = 0.1
[ToMe] ✅ Enabled in ViT: target_ratio=0.100, per_application_ratio=0.915, interval=1, num_layers=27, num_applications=26, _TOME_AVAILABLE=True
✅ Policy loaded: <class 'openpi.policies.policy.Policy'>
🔥 Warming up policy (compile + autotune)...
[ToMe Check] ✅ Token count reduced: 21 tokens (expected ~25)
[ToMe Check] ✅ Token count reduced: 21 tokens (expected ~25)
[ToMe Check] ✅ Token count reduced: 21 tokens (expected ~25)

[PI0 Inference Profiling]
  ViT (SigLIP)            : 123.81 ms
  Text Embed             : 0.83 ms
  LLM (total)    

Tasks:   0%|          | 0/10 [00:00<?, ?it/s]

[Warning]: datasets path /home/wyz/Isaac-GR00T/external_dependencies/LIBERO/libero/libero/../datasets does not exist!
[Warning]: datasets path /home/wyz/Isaac-GR00T/external_dependencies/LIBERO/libero/libero/../datasets does not exist!
[ToMe Check] ✅ Token count reduced: 21 tokens (expected ~25)
[ToMe Check] ✅ Token count reduced: 21 tokens (expected ~25)
[ToMe Check] ✅ Token count reduced: 21 tokens (expected ~25)

[PI0 Inference Profiling]
  ViT (SigLIP)            : 21.27 ms
  Text Embed             : 0.13 ms
  LLM (total)            : 66.79 ms
    ├─ prefix (KV cache): 12.62 ms
    └─ flow matching    : 54.18 ms (avg 5.42 ms/step, 10 steps)
  Flow Matching (other)  : 0.19 ms
  --------------------------------------
  Total inference        : 88.38 ms

[ToMe Check] ✅ Token count reduced: 21 tokens (expected ~25)
[ToMe Check] ✅ Token count reduced: 21 tokens (expected ~25)
[ToMe Check] ✅ Token count reduced: 21 tokens (expected ~25)

[PI0 Inference Profiling]
  ViT (SigLIP)          

Tasks:  10%|█         | 1/10 [00:10<01:37, 10.78s/it]

...

✅ Task 7: pick up the black bowl on the stove and place it on the plate
 Success rate: 0/1 = 0.0%
 Avg inference time: 111.05 ms
 Overall success: 0/8 = 0.0%

[Warning]: datasets path /home/wyz/Isaac-GR00T/external_dependencies/LIBERO/libero/libero/../datasets does not exist!
[Warning]: datasets path /home/wyz/Isaac-GR00T/external_dependencies/LIBERO/libero/libero/../datasets does not exist!
[ToMe Check] ✅ Token count reduced: 21 tokens (expected ~25)
[ToMe Check] ✅ Token count reduced: 21 tokens (expected ~25)
[ToMe Check] ✅ Token count reduced: 21 tokens (expected ~25)

[PI0 Inference Profiling]
  ViT (SigLIP)            : 24.09 ms
  Text Embed             : 0.16 ms
  LLM (total)            : 80.28 ms
    ├─ prefix (KV cache): 12.66 ms
    └─ flow matching    : 67.62 ms (avg 6.76 ms/step, 10 steps)
  Flow Matching (other)  : 0.28 ms
  --------------------------------------
  Total inference        : 104.81 ms

[ToMe Check] ✅ Token count reduced: 21 tokens (expected ~25)
[ToMe Check] 

Tasks:  90%|█████████ | 9/10 [01:30<00:10, 10.06s/it]

✅ Task 8: pick up the black bowl next to the plate and place it on the plate
 Success rate: 0/1 = 0.0%
 Avg inference time: 113.33 ms
 Overall success: 0/9 = 0.0%

[Warning]: datasets path /home/wyz/Isaac-GR00T/external_dependencies/LIBERO/libero/libero/../datasets does not exist!
[Warning]: datasets path /home/wyz/Isaac-GR00T/external_dependencies/LIBERO/libero/libero/../datasets does not exist!
[ToMe Check] ✅ Token count reduced: 21 tokens (expected ~25)
[ToMe Check] ✅ Token count reduced: 21 tokens (expected ~25)
[ToMe Check] ✅ Token count reduced: 21 tokens (expected ~25)

[PI0 Inference Profiling]
  ViT (SigLIP)            : 20.41 ms
  Text Embed             : 0.12 ms
  LLM (total)            : 64.76 ms
    ├─ prefix (KV cache): 12.61 ms
    └─ flow matching    : 52.16 ms (avg 5.22 ms/step, 10 steps)
  Flow Matching (other)  : 0.18 ms
  --------------------------------------
  Total inference        : 85.47 ms

[ToMe Check] ✅ Token count reduced: 21 tokens (expected ~25)
[ToMe Che

Tasks: 100%|██████████| 10/10 [01:40<00:00, 10.08s/it]

✅ Task 9: pick up the black bowl on the wooden cabinet and place it on the plate
 Success rate: 0/1 = 0.0%
 Avg inference time: 110.90 ms
 Overall success: 0/10 = 0.0%

🎯 Final success rate on libero_spatial: 0/10 = 0.00%
⚙️ Avg inference time over all tasks: 109.72 ± 6.76 ms


In [ ]:
import os
import sys
import time
import math
import tqdm
import imageio
import pathlib
import dataclasses
import collections
import numpy as np
import torch
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from datasets import load_dataset
from openpi.training import config as _config
from openpi.policies import policy_config
from openpi_client import image_tools

sys.path.append("/home/wyz/openpi/third_party/libero")
from libero.libero import benchmark, get_libero_path
from libero.libero.envs import OffScreenRenderEnv

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["JAX_PLATFORMS"] = "cpu"
os.environ["TORCH_DISABLE_DYNAMO"] = "1"
os.environ["PYTORCH_JIT"] = "0"

LIBERO_DUMMY_ACTION = [0.0] * 6 + [-1.0]
LIBERO_ENV_RESOLUTION = 256


# ------------------------------------------------------------
# ✅ 加载策略
# ------------------------------------------------------------
def load_policy():
    print("Loading policy checkpoint ...")
    config = _config.get_config("pi05_libero")
    ckpt = "/home/wyz/openpi/checkpoints/pi05_libero_pytorch.pt"

    def _no_compile(x, *args, **kwargs):
        return x
    torch.compile = _no_compile

    try:
        torch.jit._state.disable()
    except Exception:
        pass

    policy = policy_config.create_trained_policy(config, ckpt, pytorch_device="cuda:0")
    print("✅ Policy loaded:", type(policy))
    if hasattr(policy, "model"):
        print(" → Using model:", policy.model.__class__.__module__)
    return policy


# ------------------------------------------------------------
# ✅ 工具函数
# ------------------------------------------------------------
def _quat2axisangle(quat):
    quat = np.array(quat)
    quat = np.clip(quat, -1.0, 1.0)
    den = np.sqrt(max(1e-6, 1.0 - quat[3] ** 2))
    if math.isclose(den, 0.0):
        return np.zeros(3)
    return (quat[:3] * 2.0 * math.acos(quat[3])) / den


def _get_libero_env(task, resolution, seed):
    task_desc = task.language
    bddl_path = pathlib.Path(get_libero_path("bddl_files")) / task.problem_folder / task.bddl_file
    env_args = {"bddl_file_name": bddl_path, "camera_heights": resolution, "camera_widths": resolution}
    env = OffScreenRenderEnv(**env_args)
    env.seed(seed)
    return env, task_desc


# ------------------------------------------------------------
# ✅ 参数类
# ------------------------------------------------------------
@dataclasses.dataclass
class Args:
    task_suite_name: str = "libero_spatial"
    num_trials_per_task: int = 1       # ✅ 每个任务只跑一次
    num_steps_wait: int = 10
    replan_steps: int = 5
    resize_size: int = 224
    video_out_path: str = "./libero_videos_prune"
    seed: int = 7


# ------------------------------------------------------------
# ✅ 主评估函数
# ------------------------------------------------------------
def eval_libero_prune(args: Args):
    policy = load_policy()
    np.random.seed(args.seed)

    # ----------------------
    print("🔥 Warming up policy (compile + autotune)...")
    dummy_element = {
        "observation/image": np.zeros((args.resize_size, args.resize_size, 3), dtype=np.uint8),
        "observation/wrist_image": np.zeros((args.resize_size, args.resize_size, 3), dtype=np.uint8),
        "observation/state": np.zeros(8, dtype=np.float32),
        "prompt": "dummy task",
    }
    with torch.inference_mode():
        _ = policy.infer(dummy_element)
    torch.cuda.synchronize()
    print("✅ Warmup done, start evaluation...\n")

    # ----------------------
    benchmark_dict = benchmark.get_benchmark_dict()
    task_suite = benchmark_dict[args.task_suite_name]()
    num_tasks = task_suite.n_tasks

    pathlib.Path(args.video_out_path).mkdir(parents=True, exist_ok=True)

    # 最大步数设定
    if args.task_suite_name == "libero_spatial":
        max_steps = 220
    elif args.task_suite_name == "libero_object":
        max_steps = 280
    elif args.task_suite_name == "libero_goal":
        max_steps = 300
    else:
        max_steps = 400

    total_episodes, total_successes = 0, 0
    total_infer_times = []

    print(f"\n🔹 Running evaluation on suite: {args.task_suite_name} ({num_tasks} tasks)")

    for task_id in tqdm.tqdm(range(num_tasks), desc="Tasks"):
        task = task_suite.get_task(task_id)
        init_states = task_suite.get_task_init_states(task_id)
        init_states = init_states[:args.num_trials_per_task]

        env, desc = _get_libero_env(task, LIBERO_ENV_RESOLUTION, args.seed)
        task_episodes, task_successes = 0, 0
        task_infer_times = []

        for ep in range(args.num_trials_per_task):
            obs = env.set_init_state(init_states[ep])
            env.reset()
            action_plan = collections.deque()
            replay_imgs = []

            # 环境稳定等待
            for _ in range(args.num_steps_wait):
                obs, _, _, _ = env.step(LIBERO_DUMMY_ACTION)

            done = False
            t = 0
            while t < max_steps + args.num_steps_wait:
                try:
                    img = np.ascontiguousarray(obs["agentview_image"][::-1, ::-1])
                    wrist = np.ascontiguousarray(obs["robot0_eye_in_hand_image"][::-1, ::-1])

                    img = image_tools.convert_to_uint8(image_tools.resize_with_pad(img, args.resize_size, args.resize_size))
                    wrist = image_tools.convert_to_uint8(image_tools.resize_with_pad(wrist, args.resize_size, args.resize_size))
                    replay_imgs.append(img)

                    if not action_plan:
                        element = {
                            "observation/image": img,
                            "observation/wrist_image": wrist,
                            "observation/state": np.concatenate(
                                (obs["robot0_eef_pos"], _quat2axisangle(obs["robot0_eef_quat"]), obs["robot0_gripper_qpos"])
                            ),
                            "prompt": str(desc),
                        }

                        # ✅ 纯推理时间测量
                        with torch.inference_mode():
                            torch.cuda.synchronize()
                            t0 = time.perf_counter()
                            actions = policy.infer(element)["actions"]
                            torch.cuda.synchronize()
                            infer_time = (time.perf_counter() - t0) * 1000
                            task_infer_times.append(infer_time)

                        action_plan.extend(actions[: args.replan_steps])

                    action = action_plan.popleft()
                    obs, reward, done, info = env.step(action.tolist())

                    if done:
                        task_successes += 1
                        total_successes += 1
                        break

                    t += 1

                except Exception as e:
                    print(f"[⚠️] Exception in task {task_id}, ep {ep}: {e}")
                    break

            task_episodes += 1
            total_episodes += 1

            tag = "success" if done else "failure"
            seg = desc.replace(" ", "_")
            imageio.mimwrite(
                pathlib.Path(args.video_out_path) / f"rollout_{seg}_{tag}_ep{ep}.mp4",
                [np.asarray(x) for x in replay_imgs],
                fps=10,
            )

        # ===== 每个任务结果 =====
        sr = 100 * task_successes / max(task_episodes, 1)
        total_sr = 100 * total_successes / max(total_episodes, 1)
        avg_infer = np.mean(task_infer_times)
        total_infer_times.extend(task_infer_times)

        print(f"✅ Task {task_id}: {desc}")
        print(f" Success rate: {task_successes}/{task_episodes} = {sr:.1f}%")
        print(f" Avg inference time: {avg_infer:.2f} ms")
        print(f" Overall success: {total_successes}/{total_episodes} = {total_sr:.1f}%\n")

    # ===== 汇总结果 =====
    print("=" * 60)
    print(f"🎯 Final success rate on {args.task_suite_name}: {total_successes}/{total_episodes} = {(100 * total_successes / total_episodes):.2f}%")
    print(f"⚙️ Avg inference time over all tasks: {np.mean(total_infer_times):.2f} ± {np.std(total_infer_times):.2f} ms")
    print("=" * 60)


# ------------------------------------------------------------
# ✅ 运行入口
# ------------------------------------------------------------
if __name__ == "__main__":
    args = Args(task_suite_name="libero_spatial", num_trials_per_task=1)  # ✅ 每个任务只跑一次
    with torch.autocast(device_type='cuda', dtype=torch.bfloat16):
        eval_libero_prune(args)



In [ ]:
import os
import sys
import time
import math
import tqdm
import imageio
import pathlib
import dataclasses
import collections
import numpy as np
import torch

from datasets import load_dataset
from openpi.training import config as _config
from openpi.policies import policy_config
from openpi_client import image_tools

sys.path.append("/home/wyz/openpi/third_party/libero")
from libero.libero import benchmark, get_libero_path
from libero.libero.envs import OffScreenRenderEnv

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["JAX_PLATFORMS"] = "cpu"
os.environ["TORCH_DISABLE_DYNAMO"] = "1"
os.environ["PYTORCH_JIT"] = "0"

LIBERO_DUMMY_ACTION = [0.0] * 6 + [-1.0]
LIBERO_ENV_RESOLUTION = 256

/home/wyz/miniconda3/envs/pi0/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[robosuite WARNING] No private macro file found! (__init__.py:7)
[robosuite WARNING] It is recommended to use a private macro file (__init__.py:8)
[robosuite WARNING] To setup, run: python /home/wyz/miniconda3/envs/pi0/lib/python3.11/site-packages/robosuite/scripts/setup_macros.py (__init__.py:9)
Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.

In [ ]:
def load_policy():
    print("Loading policy checkpoint ...")
    config = _config.get_config("pi05_libero")
    ckpt = "/home/wyz/openpi/checkpoints/pi05_libero_pytorch.pt"

    def _no_compile(x, *args, **kwargs):
        return x
    torch.compile = _no_compile

    try:
        torch.jit._state.disable()
    except Exception:
        pass

    policy = policy_config.create_trained_policy(config, ckpt, pytorch_device="cuda:0")
    print("✅ Policy loaded:", type(policy))
    if hasattr(policy, "model"):
        print(" → Using model:", policy.model.__class__.__module__)
    return policy


In [ ]:

# ------------------------------------------------------------
# ✅ 工具函数
# ------------------------------------------------------------
def _quat2axisangle(quat):
    quat = np.array(quat)
    quat = np.clip(quat, -1.0, 1.0)
    den = np.sqrt(max(1e-6, 1.0 - quat[3] ** 2))
    if math.isclose(den, 0.0):
        return np.zeros(3)
    return (quat[:3] * 2.0 * math.acos(quat[3])) / den


def _get_libero_env(task, resolution, seed):
    task_desc = task.language
    bddl_path = pathlib.Path(get_libero_path("bddl_files")) / task.problem_folder / task.bddl_file
    env_args = {"bddl_file_name": bddl_path, "camera_heights": resolution, "camera_widths": resolution}
    env = OffScreenRenderEnv(**env_args)
    env.seed(seed)
    return env, task_desc


# ------------------------------------------------------------
# ✅ 参数类
# ------------------------------------------------------------
@dataclasses.dataclass
class Args:
    task_suite_name: str = "libero_spatial"
    num_trials_per_task: int = 1       # ✅ 每个任务只跑一次
    num_steps_wait: int = 10
    replan_steps: int = 5
    resize_size: int = 224
    video_out_path: str = "./libero_videos_prune"
    seed: int = 7


# ------------------------------------------------------------
# ✅ 主评估函数
# ------------------------------------------------------------
def eval_libero_prune(args: Args):
    policy = load_policy()
    np.random.seed(args.seed)

    # ----------------------
    print("🔥 Warming up policy (compile + autotune)...")
    dummy_element = {
        "observation/image": np.zeros((args.resize_size, args.resize_size, 3), dtype=np.uint8),
        "observation/wrist_image": np.zeros((args.resize_size, args.resize_size, 3), dtype=np.uint8),
        "observation/state": np.zeros(8, dtype=np.float32),
        "prompt": "dummy task",
    }
    with torch.inference_mode():
        _ = policy.infer(dummy_element)
    torch.cuda.synchronize()
    print("✅ Warmup done, start evaluation...\n")

    # ----------------------
    benchmark_dict = benchmark.get_benchmark_dict()
    task_suite = benchmark_dict[args.task_suite_name]()
    num_tasks = task_suite.n_tasks

    pathlib.Path(args.video_out_path).mkdir(parents=True, exist_ok=True)

    # 最大步数设定
    if args.task_suite_name == "libero_spatial":
        max_steps = 220
    elif args.task_suite_name == "libero_object":
        max_steps = 280
    elif args.task_suite_name == "libero_goal":
        max_steps = 300
    else:
        max_steps = 400

    total_episodes, total_successes = 0, 0
    total_infer_times = []

    print(f"\n🔹 Running evaluation on suite: {args.task_suite_name} ({num_tasks} tasks)")

    for task_id in tqdm.tqdm(range(num_tasks), desc="Tasks"):
        task = task_suite.get_task(task_id)
        init_states = task_suite.get_task_init_states(task_id)
        init_states = init_states[:args.num_trials_per_task]

        env, desc = _get_libero_env(task, LIBERO_ENV_RESOLUTION, args.seed)
        task_episodes, task_successes = 0, 0
        task_infer_times = []

        for ep in range(args.num_trials_per_task):
            obs = env.set_init_state(init_states[ep])
            env.reset()
            action_plan = collections.deque()
            replay_imgs = []

            # 环境稳定等待
            for _ in range(args.num_steps_wait):
                obs, _, _, _ = env.step(LIBERO_DUMMY_ACTION)

            done = False
            t = 0
            while t < max_steps + args.num_steps_wait:
                try:
                    img = np.ascontiguousarray(obs["agentview_image"][::-1, ::-1])
                    wrist = np.ascontiguousarray(obs["robot0_eye_in_hand_image"][::-1, ::-1])

                    img = image_tools.convert_to_uint8(image_tools.resize_with_pad(img, args.resize_size, args.resize_size))
                    wrist = image_tools.convert_to_uint8(image_tools.resize_with_pad(wrist, args.resize_size, args.resize_size))
                    replay_imgs.append(img)

                    if not action_plan:
                        element = {
                            "observation/image": img,
                            "observation/wrist_image": wrist,
                            "observation/state": np.concatenate(
                                (obs["robot0_eef_pos"], _quat2axisangle(obs["robot0_eef_quat"]), obs["robot0_gripper_qpos"])
                            ),
                            "prompt": str(desc),
                        }

                        # ✅ 纯推理时间测量
                        with torch.inference_mode():
                            torch.cuda.synchronize()
                            t0 = time.perf_counter()
                            actions = policy.infer(element)["actions"]
                            torch.cuda.synchronize()
                            infer_time = (time.perf_counter() - t0) * 1000
                            task_infer_times.append(infer_time)

                        action_plan.extend(actions[: args.replan_steps])

                    action = action_plan.popleft()
                    obs, reward, done, info = env.step(action.tolist())

                    if done:
                        task_successes += 1
                        total_successes += 1
                        break

                    t += 1

                except Exception as e:
                    print(f"[⚠️] Exception in task {task_id}, ep {ep}: {e}")
                    break

            task_episodes += 1
            total_episodes += 1

            tag = "success" if done else "failure"
            seg = desc.replace(" ", "_")
            imageio.mimwrite(
                pathlib.Path(args.video_out_path) / f"rollout_{seg}_{tag}_ep{ep}.mp4",
                [np.asarray(x) for x in replay_imgs],
                fps=10,
            )

        # ===== 每个任务结果 =====
        sr = 100 * task_successes / max(task_episodes, 1)
        total_sr = 100 * total_successes / max(total_episodes, 1)
        avg_infer = np.mean(task_infer_times)
        total_infer_times.extend(task_infer_times)

        print(f"✅ Task {task_id}: {desc}")
        print(f" Success rate: {task_successes}/{task_episodes} = {sr:.1f}%")
        print(f" Avg inference time: {avg_infer:.2f} ms")
        print(f" Overall success: {total_successes}/{total_episodes} = {total_sr:.1f}%\n")

    # ===== 汇总结果 =====
    print("=" * 60)
    print(f"🎯 Final success rate on {args.task_suite_name}: {total_successes}/{total_episodes} = {(100 * total_successes / total_episodes):.2f}%")
    print(f"⚙️ Avg inference time over all tasks: {np.mean(total_infer_times):.2f} ± {np.std(total_infer_times):.2f} ms")
    print("=" * 60)


# ------------------------------------------------------------
# ✅ 运行入口
# ------------------------------------------------------------
if __name__ == "__main__":
    args = Args(task_suite_name="libero_spatial", num_trials_per_task=1)  # ✅ 每个任务只跑一次
    with torch.autocast(device_type='cuda', dtype=torch.bfloat16):
        eval_libero_prune(args)

Loading policy checkpoint ...
✅ Policy loaded: <class 'openpi.policies.policy.Policy'>
🔥 Warming up policy (compile + autotune)...
🔍 [PI0Pytorch] 模型接收的Observation输入
  observation.state: shape=torch.Size([1, 32]), dtype=torch.float64, device=cuda:0
    values: [ 0.43457508 -0.09596702 -1.06716357 -2.76979971  0.0526286   0.31254236
 -1.08891905  1.08934945  0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.        ]
  observation.images:
    base_0_rgb: shape=torch.Size([1, 3, 224, 224]), dtype=torch.float32, device=cuda:0
      min=-1.0000, max=-1.0000, mean=-1.0000
    left_wrist_0_rgb: shape=torch.Size([1, 3, 224, 224]), dtype=torch.float32, device=cuda:0
      min=-1.0000, max=-1.0000, mean=-1.0000
    right_wrist_0_rgb: shape=torch.Size([1, 3, 224, 224]), dtype=torch.float32, device=cu

Tasks:   0%|          | 0/10 [00:00<?, ?it/s]

🔍 [PI0Pytorch] 模型接收的Observation输入
  observation.state: shape=torch.Size([1, 32]), dtype=torch.float64, device=cuda:0
    values: [-0.41394484 -0.09596702  0.86247747  0.85854427  0.05292847  0.20376065
  0.91814018 -0.9317374   0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.        ]
  observation.images:
    base_0_rgb: shape=torch.Size([1, 3, 224, 224]), dtype=torch.float32, device=cuda:0
      min=-0.9529, max=0.8667, mean=-0.0645
    left_wrist_0_rgb: shape=torch.Size([1, 3, 224, 224]), dtype=torch.float32, device=cuda:0
      min=-0.9843, max=0.8667, mean=-0.0193
    right_wrist_0_rgb: shape=torch.Size([1, 3, 224, 224]), dtype=torch.float32, device=cuda:0
      min=-1.0000, max=-1.0000, mean=-1.0000
  observation.image_masks:
    base_0_rgb: shape=torch.Size([1]), dtype=torch.bool,

Tasks:  10%|█         | 1/10 [00:04<00:40,  4.49s/it]

✅ Task 0: pick up the black bowl between the plate and the ramekin and place it on the plate
 Success rate: 1/1 = 100.0%
 Avg inference time: 78.43 ms
 Overall success: 1/1 = 100.0%

🔍 [PI0Pytorch] 模型接收的Observation输入
  observation.state: shape=torch.Size([1, 32]), dtype=torch.float64, device=cuda:0
    values: [-0.41394484 -0.09596702  0.86247747  0.85854427  0.05292847  0.20376065
  0.91814018 -0.9317374   0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.        ]
  observation.images:
    base_0_rgb: shape=torch.Size([1, 3, 224, 224]), dtype=torch.float32, device=cuda:0
      min=-0.9529, max=0.8667, mean=-0.0667
    left_wrist_0_rgb: shape=torch.Size([1, 3, 224, 224]), dtype=torch.float32, device=cuda:0
      min=-0.9843, max=0.8667, mean=0.0055
    right_wrist_0_rgb: shape=torch.Size(

...

✅ Task 7: pick up the black bowl on the stove and place it on the plate
 Success rate: 1/1 = 100.0%
 Avg inference time: 77.86 ms
 Overall success: 8/8 = 100.0%

🔍 [PI0Pytorch] 模型接收的Observation输入
  observation.state: shape=torch.Size([1, 32]), dtype=torch.float64, device=cuda:0
    values: [-0.41394484 -0.09596702  0.86247747  0.85854427  0.05292847  0.20376065
  0.91814018 -0.9317374   0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.        ]
  observation.images:
    base_0_rgb: shape=torch.Size([1, 3, 224, 224]), dtype=torch.float32, device=cuda:0
      min=-0.9529, max=0.8667, mean=-0.0657
    left_wrist_0_rgb: shape=torch.Size([1, 3, 224, 224]), dtype=torch.float32, device=cuda:0
      min=-0.9843, max=0.8667, mean=0.0053
    right_wrist_0_rgb: shape=torch.Size([1, 3, 224, 224]), dt

Tasks:  90%|█████████ | 9/10 [00:41<00:04,  4.83s/it]

[TaylorSeer] Step 00 → True Forward
[TaylorSeer] Step 01 → Forecasted (Predicted, O=0)
[TaylorSeer] Step 02 → Forecasted (Predicted, O=0)
[TaylorSeer] Step 03 → Forecasted (Predicted, O=0)
[TaylorSeer] Step 04 → Forecasted (Predicted, O=0)
[TaylorSeer] Step 05 → Forecasted (Predicted, O=0)
[TaylorSeer] Step 06 → Forecasted (Predicted, O=0)
[TaylorSeer] Step 07 → True Forward
[TaylorSeer] Step 08 → Forecasted (Predicted, O=0)
[TaylorSeer] Step 09 → Forecasted (Predicted, O=0)
[⏱ TaylorSeer-O1] EmbedPrefix: 14.92 ms | PrefixForward: 31.85 ms | Denoise: 11.56 ms | Total: 58.32 ms
✅ Task 8: pick up the black bowl next to the plate and place it on the plate
 Success rate: 1/1 = 100.0%
 Avg inference time: 84.49 ms
 Overall success: 9/9 = 100.0%

🔍 [PI0Pytorch] 模型接收的Observation输入
  observation.state: shape=torch.Size([1, 32]), dtype=torch.float64, device=cuda:0
    values: [-0.41394484 -0.09596702  0.86247747  0.85854427  0.05292847  0.20376065
  0.91814018 -0.9317374   0.          0.       

Tasks: 100%|██████████| 10/10 [00:45<00:00,  4.59s/it]

[TaylorSeer] Step 07 → True Forward
[TaylorSeer] Step 08 → Forecasted (Predicted, O=0)
[TaylorSeer] Step 09 → Forecasted (Predicted, O=0)
[⏱ TaylorSeer-O1] EmbedPrefix: 14.61 ms | PrefixForward: 31.80 ms | Denoise: 12.02 ms | Total: 58.43 ms
✅ Task 9: pick up the black bowl on the wooden cabinet and place it on the plate
 Success rate: 1/1 = 100.0%
 Avg inference time: 78.65 ms
 Overall success: 10/10 = 100.0%

🎯 Final success rate on libero_spatial: 10/10 = 100.00%
⚙️ Avg inference time over all tasks: 78.72 ± 4.33 ms


## 推理并记录Action到文件（原始LIBERO）

这个函数会对每个任务进行一次推理，打印输入和输出，并将action保存到txt文件。


In [ ]:
# 确保工具函数可用（如果前面的cell还没运行）
def _quat2axisangle(quat):
    quat = np.array(quat)
    quat = np.clip(quat, -1.0, 1.0)
    den = np.sqrt(max(1e-6, 1.0 - quat[3] ** 2))
    if math.isclose(den, 0.0):
        return np.zeros(3)
    return (quat[:3] * 2.0 * math.acos(quat[3])) / den


def _get_libero_env(task, resolution, seed):
    task_desc = task.language
    bddl_path = pathlib.Path(get_libero_path("bddl_files")) / task.problem_folder / task.bddl_file
    env_args = {"bddl_file_name": bddl_path, "camera_heights": resolution, "camera_widths": resolution}
    env = OffScreenRenderEnv(**env_args)
    env.seed(seed)
    return env, task_desc


def infer_with_normalized_actions(policy, element):
    """
    辅助函数：获取归一化之前的actions和归一化之后的actions
    返回: (normalized_actions, unnormalized_actions, normalized_state)
    """
    import jax
    from openpi.models import model as _model
    
    # 复制输入以避免修改原始数据
    inputs = jax.tree.map(lambda x: x, element)
    inputs = policy._input_transform(inputs)
    
    # 转换为PyTorch tensor
    inputs = jax.tree.map(lambda x: torch.from_numpy(np.array(x)).to(policy._pytorch_device)[None, ...], inputs)
    
    # 获取归一化后的state（模型实际看到的输入）
    normalized_state = inputs["state"][0].detach().cpu().numpy()
    
    # 获取模型输出的归一化actions（在_output_transform之前）
    observation = _model.Observation.from_dict(inputs)
    normalized_actions = policy._sample_actions(policy._pytorch_device, observation, **policy._sample_kwargs)
    if isinstance(normalized_actions, torch.Tensor):
        normalized_actions = np.asarray(normalized_actions[0, ...].detach().cpu())
    else:
        normalized_actions = np.asarray(normalized_actions[0, ...])
    
    # 应用output_transform获取反归一化后的actions
    # _output_transform期望numpy数组（与policy.infer中的处理方式一致）
    outputs = {
        "state": np.asarray(inputs["state"][0, ...].detach().cpu()),
        "actions": normalized_actions,
    }
    unnormalized_outputs = policy._output_transform(outputs)
    unnormalized_actions = np.asarray(unnormalized_outputs["actions"])
    
    return normalized_actions, unnormalized_actions, normalized_state


def infer_and_log_actions_original(task_suite_name: str = "libero_spatial", output_file: str = "inference_actions_log_original.txt"):
    """
    对每个任务进行一次推理，打印输入和输出，并将action保存到txt文件。
    使用原始的LIBERO checkpoint。
    
    Args:
        task_suite_name: 任务套件名称
        output_file: 输出txt文件路径
    """
    policy = load_policy()  # 使用原始的load_policy函数
    np.random.seed(7)
    
    # 创建输出文件
    output_path = pathlib.Path(output_file)
    output_path.parent.mkdir(parents=True, exist_ok=True)
    
    print(f"📝 输出文件: {output_path}")
    
    # Warmup
    print("🔥 Warming up policy...")
    dummy_element = {
        "observation/image": np.zeros((224, 224, 3), dtype=np.uint8),
        "observation/wrist_image": np.zeros((224, 224, 3), dtype=np.uint8),
        "observation/state": np.zeros(8, dtype=np.float32),
        "prompt": "dummy task",
    }
    with torch.inference_mode():
        _ = policy.infer(dummy_element)
    torch.cuda.synchronize()
    print("✅ Warmup done\n")
    
    # 获取任务套件
    benchmark_dict = benchmark.get_benchmark_dict()
    task_suite = benchmark_dict[task_suite_name]()
    num_tasks = task_suite.n_tasks
    
    print(f"🔹 任务套件: {task_suite_name} ({num_tasks} 个任务)")
    print("=" * 80)
    
    # 打开文件准备写入
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write("=" * 80 + "\n")
        f.write(f"推理日志 - 任务套件: {task_suite_name}\n")
        f.write(f"Checkpoint: pi05_libero (原始LIBERO)\n")
        f.write("=" * 80 + "\n\n")
        
        for task_id in range(num_tasks):
            task = task_suite.get_task(task_id)
            init_states = task_suite.get_task_init_states(task_id)
            
            env, desc = _get_libero_env(task, LIBERO_ENV_RESOLUTION, 7)
            
            # 每个任务只做一次推理
            obs = env.set_init_state(init_states[0])
            env.reset()
            
            # 环境稳定等待
            for _ in range(10):
                obs, _, _, _ = env.step(LIBERO_DUMMY_ACTION)
            
            # 准备输入
            img = np.ascontiguousarray(obs["agentview_image"][::-1, ::-1])
            wrist = np.ascontiguousarray(obs["robot0_eye_in_hand_image"][::-1, ::-1])
            
            img = image_tools.convert_to_uint8(image_tools.resize_with_pad(img, 224, 224))
            wrist = image_tools.convert_to_uint8(image_tools.resize_with_pad(wrist, 224, 224))
            
            # 构建observation - 原始机器人状态
            eef_pos = obs["robot0_eef_pos"]
            eef_quat = obs["robot0_eef_quat"]
            eef_axisangle = _quat2axisangle(eef_quat)
            gripper_qpos = obs["robot0_gripper_qpos"]
            state = np.concatenate((eef_pos, eef_axisangle, gripper_qpos))
            
            element = {
                "observation/image": img,
                "observation/wrist_image": wrist,
                "observation/state": state,
                "prompt": str(desc),
            }
            
            # 打印机器人输入（原始值）
            print(f"\n任务 {task_id}: {desc}")
            print("-" * 80)
            print("📥 机器人输入 (原始值):")
            print(f"  EEF Position (x, y, z): {eef_pos}")
            print(f"  EEF Quaternion (w, x, y, z): {eef_quat}")
            print(f"  EEF Axis-Angle (rx, ry, rz): {eef_axisangle}")
            print(f"  Gripper qpos: {gripper_qpos}")
            print(f"  State (concatenated, shape {state.shape}): {state}")
            print(f"  Image shape: {img.shape}")
            print(f"  Wrist image shape: {wrist.shape}")
            print(f"  Prompt: {desc}")
            
            # 写入文件
            f.write(f"\n任务 {task_id}: {desc}\n")
            f.write("-" * 80 + "\n")
            f.write("机器人输入 (原始值):\n")
            f.write(f"  EEF Position (x, y, z): {eef_pos}\n")
            f.write(f"  EEF Quaternion (w, x, y, z): {eef_quat}\n")
            f.write(f"  EEF Axis-Angle (rx, ry, rz): {eef_axisangle}\n")
            f.write(f"  Gripper qpos: {gripper_qpos}\n")
            f.write(f"  State (concatenated, shape {state.shape}): {state}\n")
            f.write(f"  Image shape: {img.shape}\n")
            f.write(f"  Wrist image shape: {wrist.shape}\n")
            f.write(f"  Prompt: {desc}\n")
            f.write("\n")
            
            # 推理并获取归一化前后的actions
            with torch.inference_mode():
                torch.cuda.synchronize()
                normalized_actions, unnormalized_actions, normalized_state = infer_with_normalized_actions(policy, element)
                torch.cuda.synchronize()
            
            # 打印归一化后的state（模型实际看到的输入）
            print(f"\n📊 归一化后的State (模型输入):")
            print(f"  Normalized State: {normalized_state}")
            f.write(f"\n归一化后的State (模型输入):\n")
            f.write(f"  Normalized State: {normalized_state}\n")
            
            # 打印归一化之前的actions（模型输出的原始值）
            print(f"\n📤 归一化之前的Actions (模型输出，归一化值):")
            print(f"  Normalized Actions shape: {normalized_actions.shape}")
            print(f"  Normalized Actions (完整):")
            for i, action in enumerate(normalized_actions):
                print(f"    Step {i}: {action}")
            
            # 检查归一化后的gripper值
            if normalized_actions.shape[1] >= 8:
                print(f"\n  🔍 归一化后的Gripper值 (每个step的第8维，索引7):")
                for i, action in enumerate(normalized_actions):
                    gripper_val = action[7]
                    print(f"    Step {i}: {gripper_val:.6f}")
            elif normalized_actions.shape[1] >= 7:
                print(f"\n  🔍 归一化后的Action (第7维，索引6):")
                for i, action in enumerate(normalized_actions):
                    val = action[6]
                    print(f"    Step {i}: {val:.6f}")
            
            # 写入文件
            f.write(f"\n归一化之前的Actions (模型输出，归一化值):\n")
            f.write(f"  Normalized Actions shape: {normalized_actions.shape}\n")
            f.write(f"  Normalized Actions (完整):\n")
            for i, action in enumerate(normalized_actions):
                f.write(f"    Step {i}: {action}\n")
            
            if normalized_actions.shape[1] >= 8:
                f.write(f"\n  归一化后的Gripper值 (每个step的第8维，索引7):\n")
                for i, action in enumerate(normalized_actions):
                    gripper_val = action[7]
                    f.write(f"    Step {i}: {gripper_val:.6f}\n")
            elif normalized_actions.shape[1] >= 7:
                f.write(f"\n  归一化后的Action (第7维，索引6):\n")
                for i, action in enumerate(normalized_actions):
                    val = action[6]
                    f.write(f"    Step {i}: {val:.6f}\n")
            
            # 打印反归一化后的actions（最终输出）
            print(f"\n📤 反归一化后的Actions (最终输出，原始值):")
            print(f"  Unnormalized Actions shape: {unnormalized_actions.shape}")
            print(f"  Unnormalized Actions (完整):")
            for i, action in enumerate(unnormalized_actions):
                print(f"    Step {i}: {action}")
            
            # 检查反归一化后的gripper值
            if unnormalized_actions.shape[1] >= 8:
                print(f"\n  🔍 反归一化后的Gripper值 (每个step的第8维，索引7):")
                for i, action in enumerate(unnormalized_actions):
                    gripper_val = action[7]
                    print(f"    Step {i}: {gripper_val:.6f}")
            elif unnormalized_actions.shape[1] >= 7:
                print(f"\n  🔍 反归一化后的Action (第7维，索引6):")
                for i, action in enumerate(unnormalized_actions):
                    val = action[6]
                    print(f"    Step {i}: {val:.6f}")
            
            # 写入文件
            f.write(f"\n反归一化后的Actions (最终输出，原始值):\n")
            f.write(f"  Unnormalized Actions shape: {unnormalized_actions.shape}\n")
            f.write(f"  Unnormalized Actions (完整):\n")
            for i, action in enumerate(unnormalized_actions):
                f.write(f"    Step {i}: {action}\n")
            
            if unnormalized_actions.shape[1] >= 8:
                f.write(f"\n  反归一化后的Gripper值 (每个step的第8维，索引7):\n")
                for i, action in enumerate(unnormalized_actions):
                    gripper_val = action[7]
                    f.write(f"    Step {i}: {gripper_val:.6f}\n")
            elif unnormalized_actions.shape[1] >= 7:
                f.write(f"\n  反归一化后的Action (第7维，索引6):\n")
                for i, action in enumerate(unnormalized_actions):
                    val = action[6]
                    f.write(f"    Step {i}: {val:.6f}\n")
            
            f.write("\n" + "=" * 80 + "\n")
            print("=" * 80)
    
    print(f"\n✅ 所有推理完成！结果已保存到: {output_path}")


# 运行推理
infer_and_log_actions_original(
    task_suite_name="libero_spatial",
    output_file="inference_actions_log_original.txt"
)


Loading policy checkpoint ...
✅ Policy loaded: <class 'openpi.policies.policy.Policy'>
📝 输出文件: inference_actions_log_original.txt
🔥 Warming up policy...
✅ Warmup done

[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
🔹 任务套件: libero_spatial (10 个任务)

任务 0: pick up the black bowl between the plate and the ramekin and place it on the plate
--------------------------------------------------------------------------------
📥 机器人输入 (原始值):
  EEF Position (x, y, z): [-2.08464661e-01  2.48911538e-12  1.17327948e+00]
  EEF Quaternion (w, x, y, z): [ 9.99596605e-01  2.46212834e-04 -2.84001205e-02 -6.99529628e-06]
  EEF Axis-Angle (rx, ry, rz): [ 3.14033934e+00  7.73503874e-04 -8.92220072e-02]
  Gripper qpos: [ 0.0387292  -0.03872914]
  State (concatenated, shape (8,)): [-2.08464661e-01  2.48911538e-12  1.17327948e+00  3.14033934e+00
  7.73503874e-04 -8.92220072e-02  3.87292014e-02 -3.87291413e-02]
  Image shape: (224, 224, 3)
  Wrist image shape: (224, 224, 3)
  Prompt: pick up the black bow

In [ ]:
def save_actions_only(task_suite_name: str = "libero_spatial", output_file: str = "actions_output.txt"):
    """
    只保存 action 输出到 txt 文件（简化版本）
    
    Args:
        task_suite_name: 任务套件名称
        output_file: 输出txt文件路径
    """
    policy = load_policy()
    np.random.seed(7)
    
    # 创建输出文件
    output_path = pathlib.Path(output_file)
    output_path.parent.mkdir(parents=True, exist_ok=True)
    
    print(f"📝 输出文件: {output_path}")
    
    # Warmup
    print("🔥 Warming up policy...")
    dummy_element = {
        "observation/image": np.zeros((224, 224, 3), dtype=np.uint8),
        "observation/wrist_image": np.zeros((224, 224, 3), dtype=np.uint8),
        "observation/state": np.zeros(8, dtype=np.float32),
        "prompt": "dummy task",
    }
    with torch.inference_mode():
        _ = policy.infer(dummy_element)
    torch.cuda.synchronize()
    print("✅ Warmup done\n")
    
    # 获取任务套件
    benchmark_dict = benchmark.get_benchmark_dict()
    task_suite = benchmark_dict[task_suite_name]()
    num_tasks = task_suite.n_tasks
    
    print(f"🔹 任务套件: {task_suite_name} ({num_tasks} 个任务)")
    
    # 打开文件准备写入
    with open(output_path, 'w', encoding='utf-8') as f:
        for task_id in range(num_tasks):
            task = task_suite.get_task(task_id)
            init_states = task_suite.get_task_init_states(task_id)
            
            env, desc = _get_libero_env(task, LIBERO_ENV_RESOLUTION, 7)
            
            # 每个任务只做一次推理
            obs = env.set_init_state(init_states[0])
            env.reset()
            
            # 环境稳定等待
            for _ in range(10):
                obs, _, _, _ = env.step(LIBERO_DUMMY_ACTION)
            
            # 准备输入
            img = np.ascontiguousarray(obs["agentview_image"][::-1, ::-1])
            wrist = np.ascontiguousarray(obs["robot0_eye_in_hand_image"][::-1, ::-1])
            
            img = image_tools.convert_to_uint8(image_tools.resize_with_pad(img, 224, 224))
            wrist = image_tools.convert_to_uint8(image_tools.resize_with_pad(wrist, 224, 224))
            
            # 构建observation
            eef_pos = obs["robot0_eef_pos"]
            eef_quat = obs["robot0_eef_quat"]
            eef_axisangle = _quat2axisangle(eef_quat)
            gripper_qpos = obs["robot0_gripper_qpos"]
            state = np.concatenate((eef_pos, eef_axisangle, gripper_qpos))
            
            element = {
                "observation/image": img,
                "observation/wrist_image": wrist,
                "observation/state": state,
                "prompt": str(desc),
            }
            
            # 推理获取 actions
            with torch.inference_mode():
                torch.cuda.synchronize()
                result = policy.infer(element)
                actions = result["actions"]
                torch.cuda.synchronize()
            
            # 只保存 actions 到文件
            f.write(f"Task {task_id}: {desc}\n")
            f.write(f"Actions shape: {actions.shape}\n")
            for i, action in enumerate(actions):
                f.write(f"Step {i}: {action}\n")
            f.write("\n")
            
            print(f"✅ Task {task_id}: {desc} - Actions shape: {actions.shape}")
    
    print(f"\n✅ 所有 actions 已保存到: {output_path}")


# 运行并保存 actions
save_actions_only(
    task_suite_name="libero_spatial",
    output_file="actions_output.txt"
)


Loading policy checkpoint ...
✅ Policy loaded: <class 'openpi.policies.policy.Policy'>
📝 输出文件: actions_output.txt
🔥 Warming up policy...
✅ Warmup done

[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
🔹 任务套件: libero_spatial (10 个任务)
✅ Task 0: pick up the black bowl between the plate and the ramekin and place it on the plate - Actions shape: (10, 7)
✅ Task 1: pick up the black bowl next to the ramekin and place it on the plate - Actions shape: (10, 7)
✅ Task 2: pick up the black bowl from table center and place it on the plate - Actions shape: (10, 7)
✅ Task 3: pick up the black bowl on the cookie box and place it on the plate - Actions shape: (10, 7)
✅ Task 4: pick up the black bowl in the top drawer of the wooden cabinet and place it on the plate - Actions shape: (10, 7)
✅ Task 5: pick up the black bowl on the ramekin and place it on the plate - Actions shape: (10, 7)
✅ Task 6: pick up the black bowl next to the cookie box and place it on the plate - Actions shape: (10, 7)
✅ Ta

: 